In [6]:
import cdsapi
import xarray as xr
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [7]:
today = pd.Timestamp.now().strftime('%Y-%m-%d')
now = pd.Timestamp.now().strftime('%H:%M')
if now < '17:00':
    time = '12:00'
    today = today - pd.Timedelta(days=1)
else:
    time = '00:00'


In [8]:
c = cdsapi.Client()

dataset = "cams-global-atmospheric-composition-forecasts"
request = {
    "variable": ["surface_solar_radiation_downward_clear_sky",
        "surface_solar_radiation_downwards"
    ],
    "date": [f"{today}/{today}"],
    "time": [time],
    "leadtime_hour": ["0","1","2","3","4","5","6","7","8","9","10",
        "11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26",
        "27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42",
        "43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58",
        "59","60","61","62","63","64","65","66","67","68","69","70","71","72","73","74",
        "75","76","77","78","79","80","81","82","83","84","85","86","87","88","89","90",
        "91","92","93","94","95","96","97","98","99","100","101","102","103","104","105","106",
        "107","108","109","110","111","112","113","114","115","116","117","118","119","120"],
    "type": ["forecast"],
    "data_format": "grib",
    "area": [20,95,5,106]}





result = c.retrieve(dataset,request)

result.download("cams_forecast_today.grib")


2025-07-14 22:03:05,359 INFO [2024-09-26T00:00:00] Watch our [Forum]( https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-14 22:03:06,721 INFO Request ID is 231f1d5b-57e2-42d9-9b82-a4d6fa072797
2025-07-14 22:03:06,951 INFO status has been updated to accepted
2025-07-14 22:03:40,769 INFO status has been updated to running
2025-07-14 22:06:01,352 INFO status has been updated to successful


6058114c0859637213a33dd14e75da03.grib:   0%|          | 0.00/528k [00:00<?, ?B/s]

'cams_forecast_today.grib'

In [9]:
lat = 13.736654207458994
lon = 100.53215459570426
ds = xr.open_dataset('./cams_forecast_today.grib', engine="cfgrib")
ds

Ignoring index file './cams_forecast_today.grib.5b7b6.idx' older than GRIB file


<xarray.Dataset>
Dimensions:     (step: 121, latitude: 38, longitude: 28)
Coordinates:
    number      int32 ...
    time        datetime64[ns] ...
  * step        (step) timedelta64[ns] 00:00:00 01:00:00 ... 5 days 00:00:00
    surface     float64 ...
  * latitude    (latitude) float64 19.8 19.4 19.0 18.6 18.2 ... 6.2 5.8 5.4 5.0
  * longitude   (longitude) float64 95.0 95.4 95.8 96.2 ... 105.0 105.4 105.8
    valid_time  (step) datetime64[ns] ...
Data variables:
    ssrdc       (step, latitude, longitude) float32 ...
    ssrd        (step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-07-14T22:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
df = ds.sel(latitude=lat,longitude=lon,method='nearest').to_dataframe()
df['ssrd_diff'] = (df['ssrd'] - df['ssrd'].shift(1).fillna(0))/3600

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['valid_time'] + pd.Timedelta(hours=7),y=df['ssrd_diff']/3600))
fig.show()